<a href="https://colab.research.google.com/github/Nazia-hub/Data_Science/blob/main/Generating_Meta_Description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U git+https://github.com/adbar/trafilatura.git
!pip install --upgrade pip 

!pip install spacy
!pip install transformers
!pip install bert-extractive-summarizer


from bs4 import BeautifulSoup

import csv
import os
import requests, sys
import pandas as pd
import re
import numpy as np
import trafilatura

  Cloning https://github.com/adbar/trafilatura.git to /tmp/pip-req-build-q77vld54
  Running command git clone -q https://github.com/adbar/trafilatura.git /tmp/pip-req-build-q77vld54
  Resolved https://github.com/adbar/trafilatura.git to commit 8e524da64113c529256c1d6abe5b77253111fbfd
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Attempting uninstall: chardet
    Found existing installation: chardet 3.0.4
    Uninstalling chardet-3.0.4:
      Successfully uninstalled chardet-3.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires chardet<4,>=3.0.2, but you have chardet 4.0.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trafilatura 0.9.1 requires chardet>=4.0.0; python_version >= "3.6", but you have chardet 3.0.4 which is incompatible.
htmldate 0.9.0 requires chardet>=4.0.0; python_version >= "3.6", but you have chardet 3.0.4 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# Importing my dataset as dataframe
df = pd.read_csv('/content/gdrive/My Drive/Data/metadata.csv', engine='python',header=0,
                 encoding="utf-8",index_col= False )

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df.head()

,Unnamed: 0,URL,Description,Keywords
0,0,https://lordjones.com/,Lord Jones provides the World's Finest CBD Pro...,NaN
1,1,https://www.charlottesweb.com/,Charlotte’s Web CBD ( cannabidiol ) oils conta...,NaN
2,2,https://www.thecbdistillery.com/,"Buy CBD Oil for Sale Online at CBDistillery, A...",NaN
3,3,https://medterracbd.com/,Medterra takes pride in providing our customer...,"cbd, , cbd, cbd, gel capsule, hemp, kentu..."
4,4,https://www.diamondcbd.com/,Shop from America's most trusted CBD Oil and D...,"cbd, oils, hemp oil, hemp edibles, cbd edibles..."


In [ ]:
!wget 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTGpl7KboITzqC8d-rosX_H4geyib-kHrVtVwrhM9rZSie7X35vYvC8iVJLVwGOYTemC4xm1qduMU8v/pub?gid=662239818&single=true&output=csv'

--2021-08-27 17:32:10--  https://docs.google.com/spreadsheets/d/e/2PACX-1vTGpl7KboITzqC8d-rosX_H4geyib-kHrVtVwrhM9rZSie7X35vYvC8iVJLVwGOYTemC4xm1qduMU8v/pub?gid=662239818&single=true&output=csv
Resolving docs.google.com (docs.google.com)... 74.125.204.113, 74.125.204.102, 74.125.204.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.204.113|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-00-b0-sheets.googleusercontent.com/pub/70cmver1f290kjsnpar5ku2h9g/16a70c1s4kleheofs4gvkoi7kc/1630085530000/113000363475040285402/*/e@2PACX-1vTGpl7KboITzqC8d-rosX_H4geyib-kHrVtVwrhM9rZSie7X35vYvC8iVJLVwGOYTemC4xm1qduMU8v?gid=662239818&single=true&output=csv [following]
--2021-08-27 17:32:11--  https://doc-00-b0-sheets.googleusercontent.com/pub/70cmver1f290kjsnpar5ku2h9g/16a70c1s4kleheofs4gvkoi7kc/1630085530000/113000363475040285402/*/e@2PACX-1vTGpl7KboITzqC8d-rosX_H4geyib-kHrVtVwrhM9rZSie7X35vYvC8iVJLVwGOYTemC4xm1qduMU8v?gid=662239818&

In [ ]:
df.shape
                

(11, 4)

In [ ]:
df.isnull().sum()

Unnamed: 0     0
URL            0
Description    0
Keywords       7
dtype: int64

In [ ]:
df=df.drop(columns=['Unnamed: 0'])

In [ ]:
df.head()

,URL,Description,Keywords
0,https://lordjones.com/,Lord Jones provides the World's Finest CBD Pro...,NaN
1,https://www.charlottesweb.com/,Charlotte’s Web CBD ( cannabidiol ) oils conta...,NaN
2,https://www.thecbdistillery.com/,"Buy CBD Oil for Sale Online at CBDistillery, A...",NaN
3,https://medterracbd.com/,Medterra takes pride in providing our customer...,"cbd, , cbd, cbd, gel capsule, hemp, kentu..."
4,https://www.diamondcbd.com/,Shop from America's most trusted CBD Oil and D...,"cbd, oils, hemp oil, hemp edibles, cbd edibles..."


## Running the Analysis

##### In the next cells we have one function called url_to string to get the text from a URL.

In [ ]:
# Get clean text from URL


def url_to_string(url):
  try:
    downloaded = trafilatura.fetch_url(url)
    if downloaded is not None: # assuming the download was successful
      result = trafilatura.extract(downloaded, include_tables=False, include_formatting=False, include_comments=False) 
    return result
  except ValueError as err:
    print(err)
    sys.exit(1)
    return err


   

In [ ]:
 # Create a list to store the MDs
data_x = [] 

from summarizer import Summarizer
# For each URL in the input CSV run the analysis and store the results in the list 
for i in range(len(df)):
    # Here is the URL to be analyzed
    line = df.iloc[i][0]

	# Error handling for HTTP connection problems
    try:
       body = url_to_string(line)
    except:
    	print('error while fetching', line)
    
	# BERT
    print("Summarizing URL via BERT: " + line)
    model = Summarizer()
    result = model(body, min_length=60, ratio=0.005)
    full = ''.join(result)
    print(full)

	# Storing all values into the list 
    data_x.append({"url":line, "BERT":full})

Summarizing URL via BERT: https://lordjones.com/


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Best Sellers
- Travel-Size High CBD Formula Bath Salts
Travel-Size High CBD Formula Bath Salts
- The Lord Jones Pride Sweatshirt
The Lord Jones Pride Sweatshirt
- Old Fashioned Gumdrops
Old Fashioned Gumdrops
- Royal Oil 500mg Hemp-Derived CBD
Royal Oil 500mg Hemp-Derived CBD
- Acid Mantle Repair Moisturizer
Acid Mantle Repair Moisturizer
- Hemp-Derived CBD Gel Capsules
Hemp-Derived CBD Gel Capsules
- Hemp-Derived CBD Tinctures
Hemp-Derived CBD Tinctures
- Body Lotions
Body Lotions
- Bath Salts
Bath Salts
Summarizing URL via BERT: https://www.charlottesweb.com/


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Free Shipping & Handling on all orders over $74 within the U.S
Free Shipping & Handling on all orders over $74
Free Shipping & Handling on all orders over $74 in the U.S.
Your cart is empty.
Summarizing URL via BERT: https://www.thecbdistillery.com/


...

Recommended Products
Assistance Program
To ensure access to CBD, we provide a 60% discount to veterans and individuals with disabilities, or low-income households, because everyone deserves to experience the power of CBD. Learn More
What our customers say
“The CBD Body and Massage Oil has proven to be a great tool to help my clients feel calm and relaxed during a session.
Summarizing URL via BERT: https://cbdfx.com/


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[emailprotected]
+1 (888) 991-7471
From seed to sale, our pure, single-pass CO2 extraction method produces an ultra-premium CBD oil from some of the finest organic hemp plants in the world — guaranteeing a safe and high-quality experience for maximum benefit. It is especially important for those who are pregnant, nursing, chronically ill, elderly or under the age of 18 to discuss the use of these products with a healthcare professional prior to using these products.
Summarizing URL via BERT: https://greenroads.com/


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Green Roads CBD: Award winning hemp products for a healthier you
THE GREEN ROADS difference
Green Roads has set the gold standard for CBD product quality since the earliest days of the industry. Our journey began in 2013 when Laura, after a career spent as a pharmacist, made simple CBD oils and topicals to help her friends and family find relief from everyday challenges.


### Storing data

#####In the following cells we are going to save a CSV containing for each url the summaries generated by the different algos.italicized text

In [ ]:
# Save results to the output CSV
df_new = pd.DataFrame(data_x, columns=["url", "BERT"])

In [ ]:
df_new.head()

,url,BERT
0,https://lordjones.com/,Best Sellers\n- Travel-Size High CBD Formula B...
1,https://www.charlottesweb.com/,Free Shipping & Handling on all orders over $7...
2,https://www.thecbdistillery.com/,+ FREE Shipping with CBDistillery™ Subscriptio...
3,https://medterracbd.com/,+ FREE Shipping with CBDistillery™ Subscriptio...
4,https://www.diamondcbd.com/,Chill Plus CBD & Delta-8 Extreme Tropical Mix ...


In [ ]:
from google.colab import files

# We set the variable forthe name of the CSV where we will store the new MDs 
outputcsv = 'Metadata.csv'
print("output csv name: ", outputcsv)

df_new.to_csv(outputcsv, encoding='utf-8', index=False)
print("Saving results on:", outputcsv)
files.download(outputcsv)

output csv name:  Metadata.csv
Saving results on: Metadata.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>